In [31]:
import pandas as pd
import numpy as np

In [32]:
#Take this from invoice details breakdown!
df = pd.read_excel("eleyo_files/missiongraduates_sacc_invoice_details_breakdown_08-13-2024_11-25_17235735371076493.xlsx")

In [33]:
#Copy over site column
df['Site_Filled'] = df['Site']

In [34]:
#Look for keywords in rate
keyword_to_school = {
    "Alvarado": "Alvarado Elementary School",
    "Bryant": "Bryant Elementary School",
    "Everett": "Everett Beacon Middle School",
    "Flynn": "Flynn Elementary School",
    "Marshall": "Marshall Elementary School",
    "Sanchez": "Sanchez Elementary School",
    "TECA": "Thomas Edison Charter Academy",
    "Bessie": "Bessie Carmichael Beacon TK-8",
    "Cleveland": "Cleveland Elementary School"
}

def find_substring_in_rate(rate):
    matches = [keyword_to_school[keyword] for keyword in keyword_to_school if keyword in rate]
    if len(matches) > 1:
        print(f"Error: Multiple matches found in row with Rate '{rate}': {matches}")
        return np.nan
    elif len(matches) == 1:
        return matches[0]
    return np.nan

In [43]:
df['Site_Filled'] = df.apply(
    lambda row: find_substring_in_rate(str(row['Rate'])) if pd.isna(row['Site_Filled']) else row['Site_Filled'],
    axis=1
)

In [44]:
df.head()

,Account,User,Site,Site Section,Date,End Date,Rate,Amount,Invoice,Invoice Due Date,Amount Owed,Site_Filled
0,1013.0,Beatrice Ackerman,NaN,NaN,2023-07-31,NaN,August Prorate at 50% for Beatrice Ackerman,-200.0,10488.0,2023-08-01,0.0,NaN
1,1013.0,Beatrice Ackerman,NaN,NaN,2023-08-01,NaN,Flynn Registration Fee for 23-24 School Year |...,100.0,10488.0,2023-08-01,0.0,Flynn Elementary School
2,1013.0,"Ackerman, Beatrice",Flynn Elementary School,2nd Grade | Segundo Grado,2023-08-01,2023-08-31,Regular Rate ($400.00/Month),400.0,10488.0,2023-08-01,0.0,Flynn Elementary School
3,1013.0,"Ackerman, Beatrice",Flynn Elementary School,2nd Grade | Segundo Grado,2023-09-01,2023-09-30,Regular Rate ($400.00/Month),400.0,11832.0,2023-09-01,0.0,Flynn Elementary School
4,1013.0,"Ackerman, Beatrice",Flynn Elementary School,2nd Grade | Segundo Grado,2023-10-01,2023-10-31,Regular Rate ($400.00/Month),400.0,12951.0,2023-10-01,0.0,Flynn Elementary School


In [45]:
df.to_excel("output.xlsx")

In [46]:
#Create new formatted name column
#By name
def reverse_name(name):
    if ',' in name:
        parts = name.split(',', 1)
        return ' '.join(part.strip() for part in reversed(parts))
    return name

def extract_after_for(text):
    return text.split('for ', 1)[1].strip() if 'for ' in text else text

def process_user(user, rate):
    if pd.isna(user):
        return user
    
    if not isinstance(user, str):
        return user
    if 'and' in user.lower():
        if isinstance(rate, str) and 'Prorate' in rate and 'to' in rate:
            return user
        if isinstance(rate, str) and 'Prorate' in rate:
            return extract_after_for(rate)
    if ', and' in user.lower():
        return ""
    else:
        return reverse_name(user)


# Assuming you have a DataFrame called 'df' with columns 'user' and 'rate'
# If not, you can create one like this:
# df = pd.DataFrame({
#     'user': ['Smith, John', 'Jane Doe and Jack Doe', 'Brown, Robert A.', 'Alice Johnson and Bob Johnson'],
#     'rate': ['Regular', 'Prorate - 75%', 'Prorate - 50%', 'Regular']
# })

# Apply the function to the DataFrame
df['processed_user'] = df.apply(lambda row: process_user(row['User'], row['Rate']), axis=1)

In [48]:
df.to_excel('output.xlsx')

In [49]:
#create the dictionary
# Assuming you have a DataFrame 'df' with columns 'processed_user' and 'Site'
# If not, you can create a sample one like this:
# df = pd.DataFrame({
#     'processed_user': ['John Smith', 'Jane Doe', 'John Smith', 'Alice Johnson'],
#     'Site': ['Site A', 'Site B', 'Site C', 'Site A']
# })

def create_user_site_dict(df):
    user_site_dict = {}
    
    for _, row in df.iterrows():
        user = row['processed_user']
        site = row['Site_Filled']
        
        if pd.isna(user):
            continue
        if pd.isna(site):
            continue
        if user not in user_site_dict:
            user_site_dict[user] = []
        
        if site not in user_site_dict[user]:
            user_site_dict[user].append(site)
    
    return user_site_dict

# Create the dictionary
user_to_sites = create_user_site_dict(df)